In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import json
import random
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# List of user-agent headers
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
]

# Define the scopes required for accessing Google Drive.
SCOPES = ['https://www.googleapis.com/auth/drive']

# Function to authenticate and get the Google Drive service object.
def get_drive_service():
    creds = None
    # Check if token.json exists to determine if the user has previously authenticated.
    if os.path.exists('token.json'):
        # If token.json exists, load credentials from it.
        creds = Credentials.from_authorized_user_file('token.json')

    # If credentials are missing or invalid, start the authentication flow.
    if not creds or not creds.valid:
        # Check if credentials are expired and refresh if possible.
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # If no valid credentials exist, initiate the OAuth flow.
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the refreshed or newly obtained credentials to token.json for future use.
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the Google Drive service object using the obtained credentials.
    service = build('drive', 'v3', credentials=creds)
    return service

def extract_content(url):
    # Define the API endpoint URL
    api_url = "https://u1gqxbxr66.execute-api.ap-south-1.amazonaws.com/Deploy"

    # Define the payload data to be sent in the POST request
    payload = {
        "url": f'{url}'
    }

    # Convert the payload to JSON format
    json_payload = json.dumps(payload)

    # Define the headers for the POST request
    headers = {
        'Content-Type': 'application/json'
    }

    # Send the POST request to the API endpoint
    response = requests.post(api_url, headers=headers, data=json_payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Return the extracted content as a string
        return response.text
    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch URL. Status code: {response.status_code}")
        return None

def extract_content_with_retry(url, retries=15, delay=15):
    for _ in range(retries):
        try:
            content = extract_content(url)
            return content
        except Exception as e:
            print("An error occurred:", e)
        print("Retry:", url)
        time.sleep(delay)
    return None

def extract_act_details(document_text):

    # Split the document text into sentences
    sentences = document_text.split("\n")

    # Define keywords indicating acts or bills
    act_keywords = ["Act", "Bill", "Section"]  # Add more keywords if necessary

    # Extract sentences containing act details
    act_sentences = []
    for sentence in sentences:
        for keyword in act_keywords:
            if keyword in sentence:
                act_sentences.append(sentence.strip())
                break

    act_sentences = " ".join([str(sentence) for sentence in act_sentences])

    return act_sentences

def generate_summary(document_text, num_sentences=5):
    # Remove act sentences from the document text

    # Create a parser and tokenizer
    parser = PlaintextParser.from_string(document_text, Tokenizer("english"))

    # Summarize the document without act details
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count=num_sentences)

    # Join the summary sentences
    summary_text = " ".join([str(sentence) for sentence in summary])

    return summary_text

def generate_summary_with_retry(document_text, retries=5, delay=15):
    for _ in range(retries):
        try:
            content = generate_summary(document_text)
            return content
        except Exception as e:
            print("An error occurred:", e)
        print("Retry:", document_text)
        time.sleep(delay)
    return None

if __name__ == "__main__":
  pd_data = pd.read_csv('Chennai.csv')
  failed_id = []
  failed_docs = []
  doc_id = pd_data['doc_id']
  summary_df = pd.DataFrame(columns=['doc_id', 'summary'])
  #start_index = 221892  # Define the start index
  start_index = 0

  # Obtain the Google Drive service object
  drive_service = get_drive_service()

  for index, doc_id in enumerate(doc_id[start_index:], start=start_index):
    try:
      if index % 50 == 0:
        time.sleep(5)
      summary = " "
      url = "https://indiankanoon.org/doc/{}/".format(doc_id)
      content = extract_content_with_retry(url)
      if content:
          content = content.replace("=======================================================", "")
          #print(content)

          # Create a file metadata with the desired name and MIME type
          file_metadata = {
                'name': '{}.txt'.format(doc_id),
                'parents': ['']  # Replace with the ID of the destination folder
          }
          # Create a temporary file to upload to Google Drive
          with open("temp.txt", "w", encoding="utf-8") as temp_file:
              temp_file.write(content)
          # Upload the file content to Google Drive
          media = MediaFileUpload("temp.txt", mimetype='text/plain')
          uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

          #Summary
          Summary_generated = generate_summary_with_retry(content)
          act_sentences = extract_act_details(content)
          if Summary_generated:
              summary = Summary_generated
          # Append the summary to the 'Summary' column for the respective row
          if act_sentences:
              summary = summary + act_sentences

          summary_df = summary_df._append({'index': index, 'doc_id': doc_id, 'summary': summary}, ignore_index=True)
          summary_df.to_csv(r'ch_summary_gap_11.csv', index=False)
          print("Index:", index, "Doc ID:", doc_id)
      else:
          failed_id.append(doc_id)
          failed_docs.append(url)

    except Exception as e:
      print("An error occured:", e)
      continue

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403
Error: Unable to fetch URL. Status code: 403


KeyboardInterrupt: 

In [ ]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 19.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=58809cf406aae9c15457a0a5ea9985a41e8a9bf1ba9952f2fb49994012e7f634
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fabf51175b4f0ffb437b8ab201eb07f6b3ca6a532a5536f4df8c3c0ddaf86585
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
